<a href="https://colab.research.google.com/github/adrianortega93/Paradigmas-de-Programacion/blob/main/Tarea2_Desafio_Ortega_Adrian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
    <h2>Universidad Casa Grande</h2>
    <h3>Maestría en Inteligencia Artificial y Ciencia de Datos</h3>
    <p><strong>Autor:</strong> Adrián Ortega Q.</p>
</div>
<div style="text-align: center;">
    <h3>Tarea 2 (Desafío)</h3>
</div>

In [ ]:
# ==============================================================================================================
# Tarea 2 (Desafío): Refactorización de un Script a un Sistema Orientado a Objetos con Herencia y Polimorfismo
# Curso: Paradigmas de Programación para IA y Ciencia de Datos
# Creado por: Adrián Ortega Q.
# ==============================================================================================================
# --------------------------------------------------------------------------------------------------------------
# Exportamos las librerías que vamos a necesitar (de forma separada para usar de forma global)
# --------------------------------------------------------------------------------------------------------------
import pandas as pd
from abc import ABC, abstractmethod # Importamos para la clase abstracta



<div style="text-align: center;">
    <p>En esta sección creamos la clase BookDataManager donde vamos a encapsular los métodos que serán heredados y llamados en otras clases con el fin de demostrar como funciona Herencia y Polimorfismo, no estaba dentro de lo solicitado pero se creó el Método filter_avanzado, donde además de enviar el valor a filtrar pues también se envía a que campo se aplicará ese filtro</p>
</div>

In [ ]:

# --------------------------------------------------------------------------------------------------------------
# Diseño de la Clase, en este caso la llamaremos BookDataManager
# --------------------------------------------------------------------------------------------------------------
class BookDataManager:
  def __init__(self, ruta_data):
    """
    Se inicializa Books con los datos cargados desde la url

    Args:
      ruta_data (str): La ruta remota del archivo CSV.
    """
    self.ruta_data = ruta_data
    self.df_book = pd.DataFrame()   # Se inicializa un data frame vacío
    self._load_data()     # Se define método privado para cargar la data

  @staticmethod
  def clean_column_names(df):
    """
    Método private que será usado generar una copia con los campos limpios
    """
    if df is None:   # Asegurar que no se intenta limpiar un DataFrame None
      return pd.DataFrame()

    df_resp = df.copy()
    new_campos = {col: col.lower().replace(' ', '_') for col in df_resp.columns}
    df_resp = df_resp.rename(columns = new_campos)
    return df_resp

  def _load_data(self):
    """
    Método para limpiar los campos que será usado para cargar la data
    """
    # Se usa una excepción por si algo falla, capturar el error y que el proceso continue
    try:
      df_temp = pd.read_csv(self.ruta_data)
      self.df_book = self.clean_column_names(df_temp) # Se llama al método que limpia los campos
      print("Datos cargados y procesados exitosamente.")

    except Exception as err:
      print(f"Error al cargar los datos desde {self.ruta_data}: {err}")
      self.df_book = pd.DataFrame() #Se inicia el DF vacío para que el código no se caiga

  def get_data(self):
    """
    Método público que devuelve el DataFrame completo

    Return:
      pandas.DataFrame: El DataFrame que contiene la ruta remota.
    """
    return self.df_book

  def filter_by_genre(self, genre):
    """
    Método que devuelve nuevo DataFrame aplicando filtos x género

    Args:
      genre (str): El genero que se aplicará en el filtro.

    Return:
      pandas.DataFrame: DataFrame filtrado, para otros casos devuelve vacío
    """
    # Se aplican validaciones para no afectar el flujo del proceso
    if self.df_book.empty:
      print("El DataFrame está vacío. No se puede filtrar.")
      return pd.DataFrame()

    if 'genre' not in self.df_book.columns:  # Comprobar que la columna existe
      print("La columna 'genre' no se encuentra en el DataFrame. No se puede filtrar por género.")
      return pd.DataFrame()

    # Aplicamos el filtro transformando a minúscula para que sea mas rápida capturar la coincidencia
    df_filtered = self.df_book[self.df_book['genre'].str.lower() == genre.lower()]

    if df_filtered.empty:
      print(f"No se encontraron datos para el género: '{genre}'.")

    return df_filtered

  def get_books_by_author(self, author):
    """
    Método que devuelve nuevo DataFrame aplicando filtos x autor

    Args:
      genre (str): El autor que se aplicará en el filtro.

    Return:
      pandas.DataFrame: DataFrame filtrado, para otros casos devuelve vacío
    """
    if self.df_book.empty:
        print("El DataFrame está vacío. No se puede filtrar por autor.")
        return pd.DataFrame()

    if 'author' not in self.df_book.columns: # Comprobar que la columna existe
        print("La columna 'author' no se encuentra en el DataFrame. No se puede filtrar por autor.")
        return pd.DataFrame()

    temp_author_col = self.df_book['author'].astype(str)

    # Usamos .str.contains() con case=False para hacer una búsqueda insensible a mayúsculas/minúsculas
    # y regex=False para evitar problemas si el nombre del autor tiene caracteres especiales de regex

    df_filtered = self.df_book[
        temp_author_col.str.contains(author, case=False, na=False, regex=False)
    ]
    if df_filtered.empty:
        print(f"No se encontraron datos para el autor: '{author}'.")

    return df_filtered

  # Se añade un metodo para filtrar por cualquier campo, especificando el campo

  def filter_avanzado(self, column, value):
    """
    Devuelve los libros filtrados por la columna especificada.
    """
    if self.df_book is None or self.df_book.empty:
        print("El DataFrame está vacío. No se puede filtrar.")
        return pd.DataFrame()

    if column not in self.df_book.columns:
        print(f"La columna '{column}' no existe. Columnas disponibles: {list(self.df_book.columns)}")
        return pd.DataFrame()

    if self.df_book[column].dtype == 'object':
        df_filtered = self.df_book[
            self.df_book[column].str.contains(str(value), case=False, na=False, regex=False)
        ]
    else:
        df_filtered = self.df_book[self.df_book[column] == value]

    if df_filtered.empty:
        print(f"No se encontraron resultados para {column} = '{value}'.")

    return df_filtered



<div style="text-align: center;">
    <p>En esta sección creamos la clase Abstracta BaseAnalyzer en donde se define qué métodos deben existir, pero no implementará la misma lógica al ser llamadas en las clases hijas.</p>
    <p>¿Por qué es abstracta?</p>
    <ul>
        <li>Porque no tiene una implementación concreta de analyze(), solo dice: "todas las clases hijas deberán tener su propio analyze()".
        </li>
        <li>No puedes instanciar BaseAnalyzer directamente, es solo un molde.</li>
    </ul>
    <p>¿Para qué sirve?</p>
    <ul>
        <li>Obliga a que todas las clases hijas implementen su versión personalizada de analyze().</li>
        <li>Nos asegura que todas las clases hijas se comporten igual (tienen los mismos métodos), aunque hagan cosas diferentes.</li>
    </ul>
</div>

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Diseño de la Clase Abstracta, en este caso la llamaremos BaseAnalyzer
# --------------------------------------------------------------------------------------------------------------
class BaseAnalyzer(ABC):
    """
    Clase base abstracta para realizar análisis de datos de BookDataManager.
    Define la interfaz común para todos los analizadores.
    """
    def __init__(self, data_manager):
        """
        Inicializa el analizador con una instancia de BookDataManager.

        Args:
            data_manager (BookDataManager): Una instancia del DataFrame.
        """
        # Se valida si es o no una instancia
        if not isinstance(data_manager, BookDataManager):
            raise TypeError("data_manager debe ser una instancia de BookDataManager")
        self.data_manager = data_manager

    @abstractmethod
    def analyze(self):
        """
        Método abstracto polifórmico para levantar un NotImplementedError.
        """
        raise NotImplementedError("El método 'analyze' debe ser implementado por las subclases.")

    def display_results(self, results):
        """
        Método para mostrar los resultados.

        Args:
            results (dict): Un diccionario que contiene los resultados del análisis.
        """
        print("\n--- Resultados del Análisis ---")
        if not results:
            print("No hay resultados para mostrar.")
            return

        for key, value in results.items():
            print(f"- {key}: {value}")
        print("----------------------------")


<div style="text-align: center;">
    <p>En esta sección creamos todas las clases hijas</p>
    <ul>
    <li>GeneralAnalyzer(BaseAnalyzer)</li>
    <li>GenreAnalyzer(BaseAnalyzer)</li>
    <li>AuthorAnalyzer(BaseAnalyzer)</li>
    <li>AdvancedFilterAnalyzer(BaseAnalyzer) --> No solicitada pero se añade para demostración</li>
    </ul>
    <br>
    <p><strong>Polimorfismo:</strong> significa que diferentes objetos responden al mismo método, pero con comportamientos diferentes.</p>
    <p>Para este caso en particular todos los objetos (GeneralAnalyzer, GenreAnalyzer, AuthorAnalyzer, AdvancedFilterAnalyzer) tienen un método llamado analyze() heredado, pero cada uno hace algo diferente:</p>
    <ul>
    <li>GeneralAnalyzer --> Analiza todo el dataset</li>
    <li>GenreAnalyzer --> Filtra por género</li>
    <li>AuthorAnalyzer --> Filtra por autor</li>
    <li>AdvancedFilterAnalyzer --> Filtra por cualquier columna</li>
    </ul>


In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Diseño de Clase Hija GeneralAnalyzer
# --------------------------------------------------------------------------------------------------------------
class GeneralAnalyzer(BaseAnalyzer):
    """
    Analizador general que calcula el total de libros y el rating promedio.
    """
    def analyze(self):
        """
        Calcula el total de libros y el rating promedio de todo el DF.

        Return:
            dict: Un diccionario con 'Total de Libros' y 'Rating Promedio'.
        """
        df = self.data_manager.get_data() # Usamos get_data de BookDataManager
        if df.empty:
            return {"Mensaje": "No hay datos para analizar."}

        total_books = len(df)
        # Asegurarse de que 'user_rating' sea numérico para el cálculo
        avg_rating = df['user_rating'].mean()

        return {
            "Tipo de Análisis": "General",
            "Total de Libros": total_books,
            "Rating Promedio": f"{avg_rating:.2f}"
        }

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Diseño de Clase Hija GenreAnalyzer
# --------------------------------------------------------------------------------------------------------------
class GenreAnalyzer(BaseAnalyzer):
    """
    Analizador específico por género.
    """
    def __init__(self, data_manager, genre):
        """
        Inicializa el analizador de género con el gestor de datos y el género.
        """
        super().__init__(data_manager)
        self.genre = genre

    def analyze(self):
        """
        Analiza los libros de un género específico, calculando el total y el rating promedio.

        Return:
            dict: Un diccionario con los resultados del análisis por género.
        """
        df_filtered = self.data_manager.filter_by_genre(self.genre)

        if df_filtered.empty:
            return {"Mensaje": f"No hay datos para el género '{self.genre}'."}

        total_books = len(df_filtered)
        avg_rating = df_filtered['user_rating'].mean()

        return {
            "Tipo de Análisis": f"Por Género: {self.genre}",
            "Total de Libros": total_books,
            "Rating Promedio": f"{avg_rating:.2f}"
        }


In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Diseño de Clase Hija AuthorAnalyzer
# --------------------------------------------------------------------------------------------------------------
class AuthorAnalyzer(BaseAnalyzer):
    """
    Analizador específico por autor.
    """
    def __init__(self, data_manager, author):
        """
        Inicializa el analizador de autor con el gestor de datos y el autor.
        """
        super().__init__(data_manager)
        self.author = author

    def analyze(self):
        """
        Analiza los libros de un autor específico, calculando el total y el rating promedio.

        Returns:
            dict: Un diccionario con los resultados del análisis por autor.
        """
        df_filtered = self.data_manager.get_books_by_author(self.author) # Usamos get_books_by_author de BookDataManager

        if df_filtered.empty:
            return {"Mensaje": f"No hay datos para el autor '{self.author}'."}

        total_books = len(df_filtered)
        avg_rating = df_filtered['user_rating'].mean()

        return {
            "Tipo de Análisis": f"Por Autor: {self.author}",
            "Total de Libros": total_books,
            "Rating Promedio": f"{avg_rating:.2f}"
        }

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Diseño de Clase Hija AdvancedFilterAnalyzer
# --------------------------------------------------------------------------------------------------------------
class AdvancedFilterAnalyzer(BaseAnalyzer):
    """
    Analizador avanzado que permite filtrar por cualquier columna.
    """
    def __init__(self, data_manager, column, value):
        super().__init__(data_manager)
        self.column = column
        self.value = value

    def analyze(self):
        df_filtered = self.data_manager.filter_avanzado(self.column, self.value)

        if df_filtered.empty:
            return {"Mensaje": f"No hay datos para {self.column} = '{self.value}'."}

        total_books = len(df_filtered)
        avg_rating = df_filtered['user_rating'].mean()

        return {
            "Tipo de Análisis": f"Filtro Avanzado: {self.column} = {self.value}",
            "Total de Libros": total_books,
            "Rating Promedio": f"{avg_rating:.2f}"
        }


<div style="text-align: center;">
    <p>En esta sección vemos como se instancia cada clase y donde el método analyze() tiene su propio uso para cada clase </p>
    <br>
    <p>Se crea la variable analyzers [ ] que es una lista de instancias de clases donde es llamado el método analyze() donde se demuestra que cada instancia por clase ejecuta el mismo método pero de forma diferente, aunque se colapse todo en una lista cada una tiene su propio comportamiento</p>

   


In [ ]:
# Definimos una variable con el valor de la ruta remota del archiv CSV
url_csv = "https://raw.githubusercontent.com/aiplanethub/Datasets/refs/heads/master/Amazon%20Top%2050%20Bestselling%20Books%202009%20-%202019.csv"

# 1. Crear una única instancia de BookDataManager
print("--- Inicializando BookDataManager ---")
my_book_data_manager = BookDataManager(url_csv)

#Validamos que el DataFrame no esté vacío
if my_book_data_manager.get_data().empty:
    print("No se pudieron cargar los datos. Terminando la demostración.")
else:
    # 2. Se crean las instancias de cada clase, pasándoles el my_book_data_manager
    print("\n--- Creando Instancias de Analizadores ---")
    general_analyzer = GeneralAnalyzer(my_book_data_manager)
    fiction_analyzer = GenreAnalyzer(my_book_data_manager, "Fiction")
    non_fiction_analyzer = GenreAnalyzer(my_book_data_manager, "Non Fiction")
    king_analyzer = AuthorAnalyzer(my_book_data_manager, "J.K. Rowling")
    susan_analyzer = AuthorAnalyzer(my_book_data_manager, "Suzanne Collins")
    avanzado_king_analyzer = AdvancedFilterAnalyzer(my_book_data_manager, "author", "J.K. Rowling")
    avanzado_susan_analyzer = AdvancedFilterAnalyzer(my_book_data_manager, "author", "Suzanne Collins")
    year2016_analyzer = AdvancedFilterAnalyzer(my_book_data_manager, "year", 2016)
    year2019_analyzer = AdvancedFilterAnalyzer(my_book_data_manager, "year", 2011)


    # 3. Demostración de Polimorfismo: Iterar sobre una lista de analizadores
    print("\n--- Ejecutando Análisis con Polimorfismo ---")
    analyzers = [
        general_analyzer,
        fiction_analyzer,
        non_fiction_analyzer,
        king_analyzer,
        susan_analyzer,
        avanzado_king_analyzer,
        avanzado_susan_analyzer,
        year2016_analyzer,
        year2019_analyzer
    ]

    for analyzer in analyzers:
        results = analyzer.analyze() # Aquí se demuestra el polimorfismo
        analyzer.display_results(results)